## Forest Fire

### Usage
Looks for the data file in the project subdirectory `Data` which needs to be unzipped.

```
Project
├── Data
│   ├── *unzip datafile here*
│   └── web-Stanford.txt.zip
├── Sampling Code
│   └── this file  
└── Samples
    └── results
```

Sample size is set as one of: `{5%, 10%, 15%, 20%}`. 5 samples are created as separate .csv files then (manually) moved to the `Samples` folder.

### Concept
1. Choose a node v at random
2. Generate a random number x that is geometrically distributed with mean pf/(1-pf)
3. Node v chooses x outlinks that were not yet visited (make sure we haven't gone over our sample percentage)
4. Go to those x outlinks and generate a new random x with same mean - note nodes cannot be visited a second time (so just don't include it if we hit that node)
5. If we have more samples to take just start over with a new node and do the same thing

Note to keep in mind this is a BFS algorithm - so the "to nodes" should be equal to the sample size percentage * unique nodes in the graph

Note2, we use `pf = .7` because according to this paper: [Rank degree: An efficient algorithm for graph sampling](https://www.researchgate.net/publication/310809556_Rank_degree_An_efficient_algorithm_for_graph_sampling) it gives the most representative sample


In order to run: just change the data what you want to be (must have headers: "FromNode" and "ToNode") and change sample size

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
data = pd.read_csv("../Data/web-Stanford.txt", sep="\t")

In [3]:
list_of_nodes = np.unique(np.array(data["FromNode"])) #gets the unique list of nodes
sample_size = .05*len(list_of_nodes) #so we get to hold 10% of the nodes it looks like
pf = .7*(1-.7) #this is going to be the number of nodes we select

In [4]:
def run_forest_fire4():
    nodes_seen = set()
    edges = set()
    global queue;
    queue = []
    while(len(nodes_seen) < sample_size):
        if(len(queue) == 0):
            cur_node = np.random.choice(list_of_nodes)
            queue.append(cur_node)
            nodes_seen.add(cur_node)
        def forest_fire5(node):
            if(len(nodes_seen) >= sample_size):
                return;
            else: #get a geometric random number
                outlinks_allowed = np.random.geometric(pf)
                candidates = list(data[data["FromNode"] == node]["ToNode"])
                if(outlinks_allowed <= len(candidates)):
                    candidates = random.sample(candidates, outlinks_allowed)
                for outlink in candidates:
                    if(outlink in nodes_seen):
                        edges.add((node, outlink))
                    else:
                        if(len(nodes_seen) < sample_size):
                            nodes_seen.add(outlink)
                            edges.add((node, outlink))
                            queue.append(outlink)
            queue.pop(0);     
        forest_fire5(queue[0])
    return edges

In [27]:
for i in range(1,6):
    e = run_forest_fire4()
    edgeHolder = pd.DataFrame(columns = ["FromNode", "ToNode"])
    for link in e:
        line = pd.DataFrame({"FromNode" : [link[0]], "ToNode" : [link[1]]})
        edgeHolder = pd.concat([edgeHolder, line], ignore_index = True)
    outputname = "OutputFF{}%{}.csv".format(str(int(sample_size*100)), str(i))
    edgeHolder.to_csv(outputname, index = False)

In [5]:
# Single run example

e = run_forest_fire4()

edgeHolder = pd.DataFrame(columns = ["FromNode", "ToNode"])
for link in e:
    line = pd.DataFrame({"FromNode" : [link[0]], "ToNode" : [link[1]]})
    edgeHolder = pd.concat([edgeHolder, line], ignore_index = True)

edgeHolder.to_csv("ForestFireOutput.csv")

In [6]:
edgeHolder

,FromNode,ToNode
0,180568,126067
1,247576,156983
2,49152,160833
3,45630,162664
4,99433,142871
...,...,...
44542,3374,167818
44543,89777,259455
44544,264542,81583
44545,5791,204925
